# Rich Logging and Printing

In [1]:
# default_exp logger

In [2]:
# export
from rich.console import Console
from rich.theme import Theme


def get_console(width=None):
    return Console(
                width=width,
                theme=Theme({
                    "repr.number": "bold cyan",
                    "repr.string": "bold green",
                    "logging.level.info": "dim yellow",
                    "logging.level.warning": "dim red",
                    "logging.level.exception": "bold red"}
                ),
            )

console = get_console()
# print = console.print

In [3]:
print('The Number is 128')
print({'a':1, 'b': [{1,2,3}, 'lskjdf', ], 'c':1, 'd': [{1,2,3}, 'lskjdf', ], 'ae':1, 'bf': [{1,2,3}, 'lskjdf', ], 'ag':1, 'ba': [{1,2,3}, 'lskjdf', ]})

In [4]:
print(r'\x86')

In [5]:
# export
from loguru import logger
from datetime import datetime
from fastcore.basics import patch_to
from rich.logging import RichHandler
from pathlib import Path

@patch_to(RichHandler)
def render(
    self,
    *,
    record,
    traceback,
    message_renderable: "ConsoleRenderable",
) -> "ConsoleRenderable":
    """patched the renderer to print function name as well
    """
    path = Path(record.pathname).name
    level = self.get_level_text(record)
    time_format = None if self.formatter is None else self.formatter.datefmt
    log_time = datetime.fromtimestamp(record.created)

    log_renderable = self._log_render(
        self.console,
        [message_renderable] if not traceback else [message_renderable, traceback],
        log_time=log_time,
        time_format=time_format,
        level=level,
        path=path,
        line_no=f'{record.funcName}:{record.lineno}',
        link_path=record.pathname if self.enable_link_path else None,
    )
    return log_renderable

# logger.configure(handlers=[{
#     "sink":RichHandler(rich_tracebacks=False, console=console),
#     "format":'<level>{message}</level>',
#     "backtrace": False
# }])

logger = logger

Debug = lambda x: logger.opt(depth=1).log("DEBUG", x)
Info  = lambda x: logger.opt(depth=1).log("INFO", x)
Warn  = lambda x: logger.opt(depth=1).log("WARNING", x)
Excep = lambda x: logger.opt(depth=1).log("ERROR", x)

In [6]:
Debug('TESTING {1,2,3}')
Info('TESTING {1,2,3}')
Warn('TESTING {1,2,3}')
Excep('TESTING {1,2,3}')

2021-05-27 19:39:58.862 | DEBUG    | __main__:<module>:1 - TESTING {1,2,3}
2021-05-27 19:39:58.864 | INFO     | __main__:<module>:2 - TESTING {1,2,3}
2021-05-27 19:39:58.865 | WARNING  | __main__:<module>:3 - TESTING {1,2,3}
2021-05-27 19:39:58.867 | ERROR    | __main__:<module>:4 - TESTING {1,2,3}


In [7]:
# export
def reset_logger_width(logger, width):
    return
    logger._core.handlers[1]._sink._handler.console = get_console(width=width)
    logger.info(f'reset logger\'s console width to {width}!')

In [8]:
Excep('TESTING {1,2,3}')
reset_logger_width(logger, 155)
Excep('TESTING {1,2,3}')

2021-05-27 19:39:58.883 | ERROR    | __main__:<module>:1 - TESTING {1,2,3}
2021-05-27 19:39:58.884 | ERROR    | __main__:<module>:3 - TESTING {1,2,3}


In [9]:
# exprimental
from functools import wraps
import time
def enter_exit(func):
    """
    Logs when a function
    """
    logger_ = logger.opt(depth=1)
    @wraps(func)
    def function_timer(*args, **kwargs):
        tic = time.time()
        logger_.log('DEBUG', f'Entered function `{func.__name__}`')
        o = func(*args, **kwargs)
        toc = time.time()
        logger_.log('DEBUG', f'Exiting function `{func.__name__}` after {toc-tic:.3f} seconds')
        return o
    return function_timer

In [10]:
@enter_exit
def add(x,y):
    print('sleeping...')
    time.sleep(2)
    return x+y

add(1,23)

2021-05-27 19:39:58.906 | DEBUG    | __main__:<module>:7 - Entered function `add`


2021-05-27 19:40:00.912 | DEBUG    | __main__:<module>:7 - Exiting function `add` after 2.005 seconds


24

In [12]:
import traceback
try: 1/0
except Exception as e:
    logger.warning(''.join(traceback.format_stack()))

2021-05-27 19:40:09.795 | WARNING  | __main__:<module>:4 -   File "/Users/yreddy31/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/yreddy31/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/yreddy31/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/yreddy31/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/yreddy31/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/Users/yreddy31/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/Users/yreddy31/anaconda3/lib/python3.7/asyncio/base_events.py", line 534, in run_forever
    self._run_once()
  File "/Users/yreddy31/anaconda3/lib/p